In [1]:
data_path = "src/resources/data/movie.csv"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("customer_look_alike_modelling").getOrCreate()
df = spark.read.option("header", "true").csv(data_path)
df.limit(5).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 12:07:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 12:07:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/12 12:07:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+---+-------------+-----------------+-------------+--------------------------+----------------+------+
|Age|FrequentFlyer|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedHotelOrNot|Target|
+---+-------------+-----------------+-------------+--------------------------+----------------+------+
| 34|           No|    Middle Income|            6|                        No|             Yes|     0|
| 34|          Yes|       Low Income|            5|                       Yes|              No|     1|
| 37|           No|    Middle Income|            3|                       Yes|              No|     0|
| 30|           No|    Middle Income|            2|                        No|              No|     0|
| 30|           No|       Low Income|            1|                        No|              No|     0|
+---+-------------+-----------------+-------------+--------------------------+----------------+------+



In [6]:
from pyspark.sql import functions as F
df = df.withColumnRenamed("BookedHotelOrNot", "BookedFoodOrNot")
df = df.withColumnRenamed("FrequentFlyer", "FrequentWatcher")
df = df.withColumn("Id", F.monotonically_increasing_id())
df.limit(20).show()

+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
|Age|FrequentWatcher|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|
+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
| 34|             No|    Middle Income|            6|                        No|            Yes|     0|  0|
| 34|            Yes|       Low Income|            5|                       Yes|             No|     1|  1|
| 37|             No|    Middle Income|            3|                       Yes|             No|     0|  2|
| 30|             No|    Middle Income|            2|                        No|             No|     0|  3|
| 30|             No|       Low Income|            1|                        No|             No|     0|  4|
| 27|            Yes|      High Income|            1|                        No|            Yes|     1|  5|
| 34|             No|    Mid

In [7]:
df.columns

['Age',
 'FrequentWatcher',
 'AnnualIncomeClass',
 'ServicesOpted',
 'AccountSyncedToSocialMedia',
 'BookedFoodOrNot',
 'Target',
 'Id']

In [8]:
rows_to_convert = df.columns
rows_to_convert.remove("Id")
rows_to_convert.remove("Target")
rows_to_convert

['Age',
 'FrequentWatcher',
 'AnnualIncomeClass',
 'ServicesOpted',
 'AccountSyncedToSocialMedia',
 'BookedFoodOrNot']

In [10]:
",".join(rows_to_convert)

'Age,FrequentWatcher,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedFoodOrNot'

In [11]:
df.groupBy("Target").count().show()

+------+-----+
|Target|count|
+------+-----+
|     0|  730|
|     1|  224|
+------+-----+



In [18]:
df.show()

+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
|Age|FrequentWatcher|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|
+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
| 34|             No|    Middle Income|            6|                        No|            Yes|     0|  0|
| 34|            Yes|       Low Income|            5|                       Yes|             No|     1|  1|
| 37|             No|    Middle Income|            3|                       Yes|             No|     0|  2|
| 30|             No|    Middle Income|            2|                        No|             No|     0|  3|
| 30|             No|       Low Income|            1|                        No|             No|     0|  4|
| 27|            Yes|      High Income|            1|                        No|            Yes|     1|  5|
| 34|             No|    Mid

In [19]:
df.drop("Target").toPandas().to_csv("src/resources/data/movie_master.csv", header=True, index=False)

In [15]:
from src.utils.functions import get_row_as_text, hf_embeddings, get_ars_retrieved_df
train_df = get_row_as_text(df, rows_to_convert)

In [16]:
# train_df.select("row_as_text").show(2, truncate=False)

In [17]:
# train_df.printSchema()

In [18]:
# step = 1500
# k = 4000

# texts_list = train_df.filter(F.col("Target") == 0).rdd.collect()
# # texts_list = [x[0] for x in texts_label_list]
# # texts_list
# texts_list[0]

In [19]:
# import os

# # Specify the path of the directory you want to create
# db_dir = "src/resources/embeddings/movie"

# # Create the directory
# os.makedirs(db_dir, exist_ok=True)

In [20]:
# from langchain.vectorstores import Chroma
# from src.utils.functions import hf_embeddings
# import chromadb
# import os

# client = chromadb.PersistentClient(path=db_dir)

# vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
#                  collection_metadata={"hnsw:space": "cosine"}, client=client)

In [21]:
# from langchain.vectorstores import Chroma
# vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
# for i in range(0, len(texts_list), step):
#     texts = [x.row_as_text for x in texts_list[i:i+step]]
#     metadata = [{"Id": str(x.Id), "Target": str(x.Target)} for x in texts_list[i:i+step]]
#     vdb.add_texts(texts, metadata)
#     vdb.persist()

In [22]:
# from langchain.vectorstores import Chroma
# from src.utils.functions import hf_embeddings
# import chromadb

# client = chromadb.PersistentClient(path=db_dir)

# vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
#                  collection_metadata={"hnsw:space": "cosine"}, client=client)

In [23]:
# vdb._collection.count()

In [24]:
# db_dir

In [25]:
# train_df.filter(F.col("Target") == 1).drop("row_as_text").toPandas().to_csv("src/resources/data/movie_test.csv", header=True, index=False)

In [26]:
# spark.read.option("header", "true").csv("src/resources/data/movie_test.csv").show()

In [14]:
df_input = spark.read.option("header", "true").csv("src/resources/data/movie_test.csv")

In [16]:
df_input.show()

+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
|Age|FrequentWatcher|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|
+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
| 34|            Yes|       Low Income|            5|                       Yes|             No|     1|  1|
| 27|            Yes|      High Income|            1|                        No|            Yes|     1|  5|
| 34|             No|       Low Income|            2|                       Yes|             No|     1|  7|
| 36|            Yes|      High Income|            1|                        No|             No|     1|  9|
| 28|             No|    Middle Income|            2|                        No|             No|     1| 11|
| 37|             No|       Low Income|            2|                       Yes|             No|     1| 19|
| 31|            Yes|      H

In [17]:
df_input.drop("Target").toPandas().to_csv("src/resources/data/movie_test.csv", header=True, index=False)